In [1]:
%reload_ext autoreload
%autoreload 2
import numpy
import os
import sklearn.preprocessing
import sys

from utils import get_cfg, get_index

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from classifier import classification_data

In [2]:
cfg = get_cfg()
resnet = numpy.load(os.path.join(cfg["path_base"], "cnn", "horizontal+patch=3+resnet50v2.npz"))
dataset = "resnet50v2"
orientation = "horizontal"
n_patch = 3
x = resnet["features"]
y = resnet["labels"]
n_samples = x.shape[0]
n_features = x.shape[1]
x_normalized = sklearn.preprocessing.StandardScaler().fit_transform(x)

print(x.shape, y.shape)

(1125, 2048) (1125,)


In [4]:
for pca in (128, 256, 512, 1024, -1):
    x = x_normalized
    if pca > 0:
        x = sklearn.decomposition.PCA(n_components=pca).fit_transform(x)

    if pca == -1:
        pca = n_features

    classification_data(cfg, dataset, get_index(), pca, n_samples, x, y, n_patch=n_patch, orientation=orientation)

0 DecisionTreeClassifier (900, 128) (225, 128)
1 DecisionTreeClassifier (900, 128) (225, 128)
2 DecisionTreeClassifier (900, 128) (225, 128)
3 DecisionTreeClassifier (900, 128) (225, 128)
4 DecisionTreeClassifier (900, 128) (225, 128)
best_accuracy: 53.0667, best_rule: sum

0 KNeighborsClassifier (900, 128) (225, 128)
1 KNeighborsClassifier (900, 128) (225, 128)
2 KNeighborsClassifier (900, 128) (225, 128)
3 KNeighborsClassifier (900, 128) (225, 128)
4 KNeighborsClassifier (900, 128) (225, 128)
best_accuracy: 72.0, best_rule: sum

0 MLPClassifier (900, 128) (225, 128)
1 MLPClassifier (900, 128) (225, 128)
2 MLPClassifier (900, 128) (225, 128)
3 MLPClassifier (900, 128) (225, 128)
4 MLPClassifier (900, 128) (225, 128)
best_accuracy: 78.1333, best_rule: prod

0 RandomForestClassifier (900, 128) (225, 128)
1 RandomForestClassifier (900, 128) (225, 128)
2 RandomForestClassifier (900, 128) (225, 128)
3 RandomForestClassifier (900, 128) (225, 128)
4 RandomForestClassifier (900, 128) (225, 12